# Port Performance Index Project - Main Notebook

This notebook presents the primary data analysis related to the PPI Project; see the README and other files in the [repo](https://github.com/epistemetrica/Port-Performance-Project) for full details. 

## Data Processing

AIS data is ingested from the Marine Cadastre database via the scripts found in the vessel data folder. Here, we combine the AIS data with port-level data and prepare for analysis. 

In [1]:
#prelims
import polars as pl
import pandas as pd
import geopandas as gpd

#enable string cache for polars categoricals
pl.enable_string_cache()
#display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pl.Config(tbl_rows=200);

### AIS Data

The vessel locations and status (e.g., "under way", "anchored", "moored") data include all AIS messages; for the purposes of the PPI, we only need to know when a vessel *changes* status, so all other observations are dropped. 

We also limit the data for the PPI to vessels over 100m in length. 

In [2]:
ais_gdf = (
    #read into lazyframe
    pl.scan_parquet('AIS data ingestion/data/ais_clean/*.parquet')
    #drop smaller vessels
    .filter(pl.col('length')>100)
    #sort by vessel and time
    .sort(['mmsi', 'time'])
    #indicate whether status is the same as previous row (Fill value needed to avoid status 0 evaluating as equal to false)
    .with_columns(
        status_change = (
            pl.col('status').ne(pl.col('status').shift(fill_value=20))
            .over('mmsi')
        )
    )
    #keep only new status pings
    .filter(pl.col('status_change')==True)
    #drop change col
    .drop('status_change')
    #collect to pandas df
    .collect()
    .to_pandas()
)


In [3]:

#create geopandas dataframe
ais_gdf = (
    #convert to geodataframe
    gpd.GeoDataFrame(
        ais_gdf,
        geometry=gpd.points_from_xy(ais_gdf.lon, ais_gdf.lat, crs='EPSG:4326')
    )
    #convert to WGS84 pseudo-mercator
    .to_crs(3857)
    #drop old lat lon cols
    .drop(['lat', 'lon'], axis=1)
)

### Port and Dock Data

Locations and descriptions for each dock and port come from the BTS and USACE online databases. 

In [4]:
#load port data
ports_gdf = (
    #read in shape file downloaded from BTS
    gpd.read_file('port data/Principal_Ports/Principal_Ports.shp')
    #drop unneeded columns
    .drop([
        'FID', #randomly assigned table id
        'PORT', #unknown numeric ID - not CBP or UN code
        'FOREIGN_','EXPORTS', 'IMPORTS', 'DOMESTIC' #breadown of total vol (tons)
    ], axis=1)
)
#set col names to pythonic lowercase
ports_gdf.columns = ports_gdf.columns.str.lower()

#load dock data
docks_gdf = (
    #read in shape file downloaded from USACE
    gpd.read_file('port data/Dock/Dock.shp')
    #drop unneeded columns
    .drop([
        'FID', #randomly assigned table id
        'LONGITUDE', 'LATITUDE', #already coded in 'geometry' 
        'LOCATION_D', #text description of dock location
        'STREET_ADD','ZIPCODE', #street address details
        'PSA_NAME', #statistical area name, rarely used
        'COUNTY_NAM', 'COUNTY_FIP', 'CONGRESS', 'CONGRESS_F', #county and congress info
        'MILE', 'BANK', 'LATITUDE1', 'LONGITUDE1', #redundant locaation data
        'OPERATORS', 'OWNERS', #owner info
        'PURPOSE', #long-form text description of dock uses
        'DOCK', #unknown number (not unique to each row/dock)
        'HIGHWAY_NO', 'RAILWAY_NO', 'LOCATION', #redundant location info
        'COMMODITIE', 'CONSTRUCTI','MECHANICAL', 'REMARKS', 'VERTICAL_D', 
        'DEPTH_MIN', 'DEPTH_MAX','BERTHING_L', 'BERTHING_T', 'DECK_HEIGH', 
        'DECK_HEI_1', #these are rarely used stats on construction
        'SERVICE_IN','SERVICE_TE', #rarely used indicators of data entry date 
    ], axis=1)
    #rename cols for clarity
    .rename(columns={
        'NAV_UNIT_I':'nav_unit_id',
        'NAV_UNIT_N':'nav_unit_name',
        'FACILITY_T':'facility_type',
        'CITY_OR_TO':'city',
        'STATE_POST':'state'
    })
)
#set col names to pythonic lowercase
docks_gdf.columns = docks_gdf.columns.str.lower()

### Matching

We first match each of the anchored and moored points from the AIS data with the nearest Port. 

In [7]:
stops_gdf = (
    #filter to only moorings
    ais_gdf[ais_gdf.status == 5]
    #join in nearest port to each ais message
    .sjoin_nearest(ports_gdf, how='left', exclusive=True)
    #drop unneeded cols
    .drop(['index_right', 'total'], axis=1)
    #rename cols for clarity
    .rename({'rank':'port_rank', 'type':'port_type'}, axis=1)
)

#create main df
main_gdf = (
    #merge stops back into AIS data
    ais_gdf.merge(stops_gdf, how='left')
    #sort by vessel then time of message
    .sort_values(by=['mmsi', 'time'])
)
#backfill port info except geometry (normal pandas syntax not supported for gpd geometry)
main_gdf[['port_type','port_name','port_rank']] = (
    main_gdf[['port_type','port_name','port_rank']].bfill()
)
#merge port geometries into main (NOTE backfill not supported for gpd geometry, hence the separate merge step)
main_gdf = main_gdf.merge(ports_gdf[['port_name', 'geometry']], 
                          on='port_name', how='left', suffixes=[None, '_port'])
#compute distance from message loc to port loc
main_gdf['port_dist'] = main_gdf['geometry'].distance(main_gdf['geometry_port'])

In [8]:
#inspect results
main_gdf.head()

,mmsi,time,speed,course,heading,status,vessel_name,vessel_type,imo,length,width,draft,cargo,geometry,port_type,port_name,port_rank,geometry_port,port_dist
0,102810,2015-07-26 02:13:42,0.0,13.6,14.0,0.0,SINGLEVESSEL,79.0,4711.0,130.0,16.0,0.0,79.0,POINT (-13135627.556 3988362.673),C,"Houston Port Authority, TX",1.0,POINT (-10608043.822 3471073.280),2.579974e+06
1,30474600,2015-02-15 19:28:50,12.3,317.0,318.0,0.0,BBC PLATA,72.0,9291975.0,138.0,21.0,6.2,70.0,POINT (-10223786.862 3147923.201),C,"Houston Port Authority, TX",1.0,POINT (-10608043.822 3471073.280),5.020751e+05
2,30474600,2015-02-16 13:10:16,0.5,254.0,189.0,1.0,BBC PLATA,72.0,9291975.0,138.0,21.0,5.9,72.0,POINT (-10529147.358 3393942.197),C,"Houston Port Authority, TX",1.0,POINT (-10608043.822 3471073.280),1.103352e+05
3,30474600,2015-02-16 21:44:44,5.3,23.0,15.0,0.0,BBC PLATA,72.0,9291975.0,138.0,21.0,5.9,72.0,POINT (-10528695.400 3393776.506),C,"Houston Port Authority, TX",1.0,POINT (-10608043.822 3471073.280),1.107744e+05
4,30474600,2015-02-17 05:18:24,0.0,0.0,51.0,5.0,BBC PLATA,72.0,9291975.0,138.0,21.0,5.9,72.0,POINT (-10602713.956 3467123.195),C,"Houston Port Authority, TX",1.0,POINT (-10608043.822 3471073.280),6.634051e+03
